In [1]:
import sys
sys.path.append("../")

In [2]:
import os

import pandas as pd
import joblib

from sklearn.ensemble import RandomForestRegressor

import os.path as op
import random

import matplotlib.pyplot as plt
import yaml

from agents.matchmanager import MatchManager
from agents.ml.regressionMulti import RegressionMultiAgent
from agents import GreedyAgent,RandomAgent
from agents.matchmanager import MatchManager
from agents.ml.classifier import ClassifierAgent
from agents.ml.regression import RegressionAgent
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor

from core.const import RED, BLUE
from scenarios import scenarioJunction

from core.vectors import vectorAction, vectorActionInfo, vectorBoard, vectorBoardInfo
from core.game import GameBoard, GameState, vectorState, vectorStateInfo


import warnings

%matplotlib inline
plt.rcParams['figure.figsize'] = (12,4)

In [3]:
with open(op.join('.', '../logger.config.yaml'), 'r') as stream:
    config = yaml.load(stream, Loader=yaml.FullLoader)
# logging.config.dictConfig(config)

In [4]:
df = pd.read_pickle('../data.2021-03-01.scenarioJunction.pkl.gz')\
        .drop(['meta_scenario', 'meta_p_red', 'meta_p_blue', 'meta_seed','meta_id'], axis=1, errors="ignore")

In [5]:
df.dropna(axis=1, how='all',inplace=True)

In [33]:
df.columns

Index(['turn', 'red_figure_0_index', 'red_figure_0_kind', 'red_figure_0_end',
       'red_figure_0_int_atk', 'red_figure_0_int_def', 'red_figure_0_move',
       'red_figure_0_load', 'red_figure_0_hp', 'red_figure_0_hp_max',
       ...
       'blue_goal_GoalEliminateOpponent_score',
       'blue_goal_GoalEliminateOpponent_check',
       'blue_goal_GoalDefendPoint_score', 'blue_goal_GoalDefendPoint_check',
       'blue_goal_GoalMaxTurn_score', 'blue_goal_GoalMaxTurn_check',
       'action_move_destination_protection_level',
       'action_move_destination_move_cost_infantry',
       'action_move_destination_move_cost_vehicle', 'winner'],
      dtype='object', length=373)

In [40]:
df[['red_figure_0_hp','red_figure_0_load','red_figure_0_int_def']].sample(n=10)

,red_figure_0_hp,red_figure_0_load,red_figure_0_int_def
21,0,1,1
27,1,1,1
15,1,1,1
11,0,1,0
16,0,1,1
18,0,1,1
14,1,1,1
12,1,1,1
7,0,1,0
18,0,1,1


In [16]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df.sample(n=5))

,turn,red_figure_0_index,red_figure_0_kind,red_figure_0_end,red_figure_0_int_atk,red_figure_0_int_def,red_figure_0_move,red_figure_0_load,red_figure_0_hp,red_figure_0_hp_max,red_figure_0_bonus,red_figure_0_activated,red_figure_0_responded,red_figure_0_attacked,red_figure_0_moved,red_figure_0_passed,red_figure_0_killed,red_figure_0_hit,red_figure_0_attacked_by,red_figure_0_can_transport,red_figure_0_transport_capacity,red_figure_0_len_transporting,red_figure_0_transported_by,red_figure_0_positionX,red_figure_0_positionY,red_figure_0_positionZ,red_figure_0_stat_value,red_figure_0_stat_Hidden,red_figure_0_stat_Loaded,red_figure_0_stat_Noeffect,red_figure_0_stat_Inmotion,red_figure_0_stat_Upstairs,red_figure_0_stat_Underfire,red_figure_0_stat_Cutoff,red_figure_0_weapon_CA,red_figure_0_weapon_AR,red_figure_0_weapon_MG,red_figure_0_weapon_AT,red_figure_0_weapon_MT,red_figure_0_weapon_GR,red_figure_0_weapon_SM,red_figure_0_weapon_SR,red_figure_0_defense_basic,red_figure_0_defense_smoke,red_figure_0_defense_antitank,red_figure_1_index,red_figure_1_kind,red_figure_1_end,red_figure_1_int_atk,red_figure_1_int_def,red_figure_1_move,red_figure_1_load,red_figure_1_hp,red_figure_1_hp_max,red_figure_1_bonus,red_figure_1_activated,red_figure_1_responded,red_figure_1_attacked,red_figure_1_moved,red_figure_1_passed,red_figure_1_killed,red_figure_1_hit,red_figure_1_attacked_by,red_figure_1_can_transport,red_figure_1_transport_capacity,red_figure_1_len_transporting,red_figure_1_transported_by,red_figure_1_positionX,red_figure_1_positionY,red_figure_1_positionZ,red_figure_1_stat_value,red_figure_1_stat_Hidden,red_figure_1_stat_Loaded,red_figure_1_stat_Noeffect,red_figure_1_stat_Inmotion,red_figure_1_stat_Upstairs,red_figure_1_stat_Underfire,red_figure_1_stat_Cutoff,red_figure_1_weapon_CA,red_figure_1_weapon_AR,red_figure_1_weapon_MG,red_figure_1_weapon_AT,red_figure_1_weapon_MT,red_figure_1_weapon_GR,red_figure_1_weapon_SM,red_figure_1_weapon_SR,red_figure_1_defense_basic,red_figure_1_defense_smoke,red_figure_1_defense_antitank,red_figure_2_index,red_figure_2_kind,red_figure_2_end,red_figure_2_int_atk,red_figure_2_int_def,red_figure_2_move,red_figure_2_load,red_figure_2_hp,red_figure_2_hp_max,red_figure_2_bonus,red_figure_2_activated,red_figure_2_responded,red_figure_2_attacked,red_figure_2_moved,red_figure_2_passed,red_figure_2_killed,red_figure_2_hit,red_figure_2_attacked_by,red_figure_2_can_transport,red_figure_2_transport_capacity,red_figure_2_len_transporting,red_figure_2_transported_by,red_figure_2_positionX,red_figure_2_positionY,red_figure_2_positionZ,red_figure_2_stat_value,red_figure_2_stat_Hidden,red_figure_2_stat_Loaded,red_figure_2_stat_Noeffect,red_figure_2_stat_Inmotion,red_figure_2_stat_Upstairs,red_figure_2_stat_Underfire,red_figure_2_stat_Cutoff,red_figure_2_weapon_CA,red_figure_2_weapon_AR,red_figure_2_weapon_MG,red_figure_2_weapon_AT,red_figure_2_weapon_MT,red_figure_2_weapon_GR,red_figure_2_weapon_SM,red_figure_2_weapon_SR,red_figure_2_defense_basic,red_figure_2_defense_smoke,red_figure_2_defense_antitank,blue_figure_0_index,blue_figure_0_kind,blue_figure_0_end,blue_figure_0_int_atk,blue_figure_0_int_def,blue_figure_0_move,blue_figure_0_load,blue_figure_0_hp,blue_figure_0_hp_max,blue_figure_0_bonus,blue_figure_0_activated,blue_figure_0_responded,blue_figure_0_attacked,blue_figure_0_moved,blue_figure_0_passed,blue_figure_0_killed,blue_figure_0_hit,blue_figure_0_attacked_by,blue_figure_0_can_transport,blue_figure_0_transport_capacity,blue_figure_0_len_transporting,blue_figure_0_transported_by,blue_figure_0_positionX,blue_figure_0_positionY,blue_figure_0_positionZ,blue_figure_0_stat_value,blue_figure_0_stat_Hidden,blue_figure_0_stat_Loaded,blue_figure_0_stat_Noeffect,blue_figure_0_stat_Inmotion,blue_figure_0_stat_Upstairs,blue_figure_0_stat_Underfire,blue_figure_0_stat_Cutoff,blue_figure_0_weapon_CA,blue_figure_0_weapon_AR,blue_figure_0_weapon_MG,blue_figure_0_weapon_AT,blue_figure_0_weapon_MT,blue_figure_0_weapon_GR,blue_figure_0

In [8]:
df.head()

,turn,red_figure_0_index,red_figure_0_kind,red_figure_0_end,red_figure_0_int_atk,red_figure_0_int_def,red_figure_0_move,red_figure_0_load,red_figure_0_hp,red_figure_0_hp_max,...,blue_goal_GoalEliminateOpponent_score,blue_goal_GoalEliminateOpponent_check,blue_goal_GoalDefendPoint_score,blue_goal_GoalDefendPoint_check,blue_goal_GoalMaxTurn_score,blue_goal_GoalMaxTurn_check,action_move_destination_protection_level,action_move_destination_move_cost_infantry,action_move_destination_move_cost_vehicle,winner
0,0,0,2,0,6,0,7,1,1,1,...,3.0,False,3.977778,False,0.909091,False,-1,-1,-1,red
1,0,0,2,0,6,0,7,1,1,1,...,3.0,False,3.888889,False,0.909091,False,-1,-1,-1,red
2,0,0,2,0,6,0,7,1,1,1,...,3.0,False,3.888889,False,0.909091,False,-1,-1,-1,red
3,0,0,2,0,6,0,7,1,1,1,...,3.0,False,11.933333,False,0.909091,False,-1,-1,-1,red
4,0,0,2,0,6,0,7,1,1,1,...,3.0,False,11.933333,False,0.909091,False,-1,-1,-1,red


In [17]:
df_red = df[df['action_team'] == 'red'].copy().drop('action_team', axis=1)
df_blue = df[df['action_team'] == 'blue'].copy().drop('action_team', axis=1)

In [19]:
df_red['label'] = df_red['winner'].apply(lambda x: 1 if x == 'red' else -1)
df_blue['label'] = df_blue['winner'].apply(lambda x: 1 if x == 'blue' else -1)

In [42]:
df_red[['turn','red_figure_0_hp','red_figure_0_load','red_figure_0_int_def','blue_figure_0_hp','blue_figure_0_load','blue_figure_0_int_def','winner','label']].sample(n=5)

,turn,red_figure_0_hp,red_figure_0_load,red_figure_0_int_def,blue_figure_0_hp,blue_figure_0_load,blue_figure_0_int_def,winner,label
26,3,1,1,1,0,1,1,blue,-1
7,0,0,1,0,1,1,0,blue,-1
11,0,0,1,0,1,1,0,blue,-1
12,1,1,1,1,0,1,1,red,1
8,0,1,1,0,1,1,0,red,1


In [44]:
df_blue[['turn','red_figure_0_hp','red_figure_0_load','red_figure_0_int_def','blue_figure_0_hp','blue_figure_0_load','blue_figure_0_int_def','winner','label']].sample(n=5)

,turn,red_figure_0_hp,red_figure_0_load,red_figure_0_int_def,blue_figure_0_hp,blue_figure_0_load,blue_figure_0_int_def,winner,label
1,0,1,1,0,1,1,0,red,-1
9,0,1,1,0,1,1,0,red,-1
13,1,0,1,1,1,1,1,blue,1
1,0,1,1,0,1,1,0,red,-1
10,0,0,1,0,1,1,0,blue,1


In [ ]:
df_red_move=df_red.loc[((df_red['action_type_Move'] == True)|(df_red['action_type_MoveLoadInto'] == True))].copy()
df_red_attack=df_red.loc[((df_red['action_type_Attack'] == True)| (df_red['action_type_AttackGround'] == True))].copy()
df_red_pass=df_red.loc[((df_red['action_type_Pass'] == True) | (df_red['action_type_PassFigure'] == True) |(df_red['action_type_PassTeam'] == True) )].copy()
df_red_response=df_red.loc[((df_red['action_type_AttackRespond'] == True) | (df_red['action_type_PassRespond'] == True))].copy()



df_red_move=df_red.loc[((df_red['action_type_Move'] == True)|(df_red['action_type_MoveLoadInto'] == True))].copy()
df_red_attack=df_red.loc[((df_red['action_type_Attack'] == True)| (df_red['action_type_AttackGround'] == True) |(df_red['action_type_AttackRespond'] == True))].copy()
df_red_pass=df_red.loc[((df_red['action_type_Pass'] == True)| (df_red['action_type_PassFigure'] == True) |(df_red['action_type_PassTeam'] == True) | (df_red['action_type_PassRespond'] == True))].copy()



In [ ]:
df_blue_move=df_blue.loc[((df_blue['action_type_Move'] == True)|(df_blue['action_type_MoveLoadInto'] == True))].copy()
df_blue_attack=df_blue.loc[((df_blue['action_type_Attack'] == True)| (df_blue['action_type_AttackGround'] == True) |(df_blue['action_type_AttackRespond'] == True))].copy()
df_blue_pass=df_blue.loc[((df_blue['action_type_Pass'] == True)| (df_blue['action_type_PassFigure'] == True) |(df_blue['action_type_PassTeam'] == True) | (df_blue['action_type_PassRespond'] == True))].copy()

In [ ]:
X_red = df_red.drop(['winner', 'label'], axis=1)
y_red = df_red['label']

X_blue = df_blue.drop(['winner', 'label'], axis=1)
y_blue = df_blue['label']

In [ ]:
X_red_move = df_red_move.drop(['winner', 'label'], axis=1)
y_red_move = df_red_move['label']

X_blue_move = df_blue_move.drop(['winner', 'label'], axis=1)
y_blue_move = df_blue_move['label']

In [ ]:
X_red_attack = df_red_attack.drop(['winner', 'label'], axis=1)
y_red_attack  = df_red_attack['label']

X_blue_attack  = df_blue_attack.drop(['winner', 'label'], axis=1)
y_blue_attack  = df_blue_attack['label']

In [ ]:
X_red_pass = df_red_pass.drop(['winner', 'label'], axis=1)
y_red_pass  = df_red_pass['label']

X_blue_pass  = df_blue_pass.drop(['winner', 'label'], axis=1)
y_blue_pass  = df_blue_pass['label']

In [ ]:
X = pd.concat([X_red, X_blue])
y = pd.concat([y_red, y_blue])

In [ ]:
rfr_red = RandomForestRegressor()
rfr_blue = RandomForestRegressor()
rfr = RandomForestRegressor()

rfr_red.fit(X_red, y_red)
rfr_blue.fit(X_blue, y_blue)
rfr.fit(X, y)

joblib.dump(rfr_red, '../models/Junction_RandomForestRegressor_red_20210301.joblib')
joblib.dump(rfr_blue, '../models/Junction_RandomForestRegressor_blue_20210301.joblib')
joblib.dump(rfr, '../models/Junction_RandomForestRegressor_20210301.joblib')

In [ ]:
rfc_red = RandomForestClassifier()
rfc_blue = RandomForestClassifier()
rfc = RandomForestClassifier()

rfc_red.fit(X_red, y_red)
rfc_blue.fit(X_blue, y_blue)
rfc.fit(X, y)

joblib.dump(rfc_red, '../models/Junction_RandomForestClassifier_red_20210301.joblib')
joblib.dump(rfc_blue, '../models/Junction_RandomForestClassifier_blue_20210301.joblib')
joblib.dump(rfc, '../models/Junction_RandomForestClassifier_20210301.joblib')

In [ ]:
rfr_red_move = RandomForestRegressor()
rfr_blue_move = RandomForestRegressor()
rfr_red_attack = RandomForestRegressor()
rfr_blue_attack = RandomForestRegressor()
rfr_red_pass = RandomForestRegressor()
rfr_blue_pass = RandomForestRegressor()
rfr = RandomForestRegressor()

rfr_red_move.fit(X_red_move, y_red_move)
rfr_red_attack.fit(X_red_attack, y_red_attack)
rfr_red_pass.fit(X_red_pass, y_red_pass)

rfr_blue_move.fit(X_blue_move, y_blue_move)
rfr_blue_attack.fit(X_blue_attack, y_blue_attack)
rfr_blue_pass.fit(X_blue_pass, y_blue_pass)


rfr.fit(X, y)

joblib.dump(rfr_red_move, '../models/Junction_RandomForestRegressor_red_move_20210301.joblib')
joblib.dump(rfr_red_attack, '../models/Junction_RandomForestRegressor_red_attack_20210301.joblib')
joblib.dump(rfr_red_pass, '../models/Junction_RandomForestRegressor_red_pass_20210301.joblib')


joblib.dump(rfr_blue_move, '../models/Junction_RandomForestRegressor_blue_move_20210301.joblib')
joblib.dump(rfr_blue_attack, '../models/Junction_RandomForestRegressor_blue_attack_20210301.joblib')
joblib.dump(rfr_blue_pass, '../models/Junction_RandomForestRegressor_blue_pass_20210301.joblib')

